In [2]:

!pip install requests beautifulsoup4


In [3]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = requests.Session()
retry = Retry(connect=3, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

try:
    response = session.get("https://www.thehindu.com/", timeout=10)
    print(response.status_code)
except requests.exceptions.RequestException as e:
    print("Error:", e)


200


In [4]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_eng Averaged Perceptron Tagger (JSON)
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] averaged_perceptron_tagger_rus Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] bcp47............... BCP-47 Language Tags
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ 

True

In [5]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
txt = '''The official website of Maulana Azad National Institute of Technology (MANIT) often hosts important circulars, event announcements, academic calendars, and examination schedules.
These updates are typically published on dedicated pages such as "Notices", "Academic", or "Events".
By crawling the MANIT website, students can programmatically collect this data and make it searchable.
Regularly checking these pages ensures students stay up to date with institutional announcements.'''

tokens = word_tokenize(txt.lower())
print(tokens)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...


['the', 'official', 'website', 'of', 'maulana', 'azad', 'national', 'institute', 'of', 'technology', '(', 'manit', ')', 'often', 'hosts', 'important', 'circulars', ',', 'event', 'announcements', ',', 'academic', 'calendars', ',', 'and', 'examination', 'schedules', '.', 'these', 'updates', 'are', 'typically', 'published', 'on', 'dedicated', 'pages', 'such', 'as', '``', 'notices', "''", ',', '``', 'academic', "''", ',', 'or', '``', 'events', "''", '.', 'by', 'crawling', 'the', 'manit', 'website', ',', 'students', 'can', 'programmatically', 'collect', 'this', 'data', 'and', 'make', 'it', 'searchable', '.', 'regularly', 'checking', 'these', 'pages', 'ensures', 'students', 'stay', 'up', 'to', 'date', 'with', 'institutional', 'announcements', '.']


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
import requests

url = "https://www.thehindu.com/"
response = requests.get(url)


with open("manit_page.html", "w", encoding='utf-8') as file:
    file.write(response.text)

print("Page downloaded successfully!")


Page downloaded successfully!


In [19]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque, defaultdict
import hashlib
import json

DOWNLOAD_DIR = "downloaded_pages"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

url_map = {}

def is_valid_url(url):
    parsed = urlparse(url)
    return parsed.scheme in ["http", "https"] and parsed.netloc

def get_domain(url):
    return urlparse(url).netloc

def url_to_filename(url):
    hash_name = hashlib.md5(url.encode()).hexdigest()
    return os.path.join(DOWNLOAD_DIR, f"{hash_name}.html")

def crawl(seed_urls, max_pages_per_site):
    visited = set()
    queue = deque(seed_urls)

    domain_counts = defaultdict(int)

    while queue:
        url = queue.popleft()
        domain = get_domain(url)

        if not is_valid_url(url):
            continue

        if url in visited:
            continue


        if domain_counts[domain] >= max_pages_per_site:
            continue

        print(f"Downloading: {url}")

        try:
            response = requests.get(url, timeout=5)
            if 'text/html' not in response.headers.get('Content-Type', ''):
                continue
        except Exception as e:
            print(f"Error downloading {url}: {e}")
            continue

        filename = url_to_filename(url)
        with open(filename, "w", encoding="utf-8") as f:
            f.write(response.text)

        short_filename = os.path.basename(filename)
        url_map[short_filename] = url

        visited.add(url)
        domain_counts[domain] += 1

        print(f"Saved ({domain_counts[domain]}/{max_pages_per_site}) from {domain}")

        soup = BeautifulSoup(response.text, "html.parser")
        for link_tag in soup.find_all("a", href=True):
            link = urljoin(url, link_tag["href"])
            if link not in visited:
                queue.append(link)


        if all(domain_counts[get_domain(seed)] >= max_pages_per_site for seed in seed_urls):
            break

    with open("url_map.json", "w", encoding="utf-8") as f:
        json.dump(url_map, f, indent=2)

    print("\n✔ Crawling complete")
    for domain, count in domain_counts.items():
        print(f"{domain}: {count} pages")


In [20]:
seed = [
    "https://www.thehindu.com/",
    "https://timesofindia.indiatimes.com/",
    "https://www.deccanchronicle.com/",
    "https://www.bbc.com/"
]

crawl(seed, max_pages_per_site=100)


Downloading: https://www.thehindu.com/
Saved (1/100) from www.thehindu.com
Downloading: https://timesofindia.indiatimes.com/
Saved (1/100) from timesofindia.indiatimes.com
Downloading: https://www.deccanchronicle.com/
Saved (1/100) from www.deccanchronicle.com
Downloading: https://www.bbc.com/
Saved (1/100) from www.bbc.com
Downloading: https://epaper.thehindu.com/reader?utm_source=Hindu&utm_medium=Menu&utm_campaign=Header&_gl=1*1u4uvwa*_gcl_au*NDk4ODM2Nzc2LjE3MjUwNzc4OTIuNTIxNjMwNzI4LjE3MjU0NDM0NDkuMTcyNTQ0MzQ0OQ..
Saved (1/100) from epaper.thehindu.com
Downloading: https://www.thehindu.com/myaccount/
Saved (2/100) from www.thehindu.com
Downloading: https://www.thehindu.com/premium/ebook/
Saved (3/100) from www.thehindu.com
Downloading: https://www.thehindu.com/subscription/
Saved (4/100) from www.thehindu.com
Downloading: https://www.thehindu.com/search/
Saved (5/100) from www.thehindu.com
Downloading: https://www.thehindu.com/news/national/
Saved (6/100) from www.thehindu.com
Downlo

In [21]:
import os
import json
from bs4 import BeautifulSoup

HTML_DIR = "downloaded_pages"
TEXT_DIR = "text_pages"
URL_MAP_FILE = "url_map.json"


os.makedirs(TEXT_DIR, exist_ok=True)


with open(URL_MAP_FILE, "r", encoding="utf-8") as f:
    url_map = json.load(f)

def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, "html.parser")


    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()


    text = soup.get_text(separator="\n")


    lines = [line.strip() for line in text.splitlines()]
    cleaned_text = "\n".join(line for line in lines if line)
    return cleaned_text

for filename in os.listdir(HTML_DIR):
    if filename.endswith(".html"):
        html_path = os.path.join(HTML_DIR, filename)
        with open(html_path, "r", encoding="utf-8") as html_file:
            html_content = html_file.read()

        plain_text = extract_text_from_html(html_content)


        original_url = url_map.get(filename, "URL_NOT_FOUND")


        txt_filename = filename.replace(".html", ".txt")
        txt_path = os.path.join(TEXT_DIR, txt_filename)
        with open(txt_path, "w", encoding="utf-8") as txt_file:
            txt_file.write(f"URL: {original_url}\n\n{plain_text}")

        print(f"✅ Extracted text from {filename} → {txt_filename}")

print("\n🎉 All HTML files have been processed into plain text with URLs included!")


✅ Extracted text from a59cab24820587a0092636ccfc93ced3.html → a59cab24820587a0092636ccfc93ced3.txt
✅ Extracted text from fbddb6825848d163d484313e1861dbf2.html → fbddb6825848d163d484313e1861dbf2.txt
✅ Extracted text from f96b51adec2df178c7d09ce7c1a3e0a4.html → f96b51adec2df178c7d09ce7c1a3e0a4.txt
✅ Extracted text from 87e28d637c8b9425887bcce2de58701a.html → 87e28d637c8b9425887bcce2de58701a.txt
✅ Extracted text from beab19788dcd505c152601d62e16fcd0.html → beab19788dcd505c152601d62e16fcd0.txt
✅ Extracted text from 7e8cf390875079db73c023a1bc64e6ba.html → 7e8cf390875079db73c023a1bc64e6ba.txt
✅ Extracted text from 645261d29072ad2723cb8863dbcc489d.html → 645261d29072ad2723cb8863dbcc489d.txt
✅ Extracted text from 9c30be56c1a686b14882835d156cd457.html → 9c30be56c1a686b14882835d156cd457.txt
✅ Extracted text from f67700ab98f466850079babc7b390b9f.html → f67700ab98f466850079babc7b390b9f.txt
✅ Extracted text from d6f109d7b60060e74df0fa58a24fae1e.html → d6f109d7b60060e74df0fa58a24fae1e.txt
✅ Extracte

In [12]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



stop_words = set(stopwords.words('english'))


filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

print("Filtered Tokens (without stopwords):")
print(*filtered_tokens)
print(len(filtered_tokens))


Filtered Tokens (without stopwords):
mumbai news today latest mumbai news hindu december 23 2025 login account ebooks subscribe search india world movies sport data health opinion science business premium live news section news india world states cities topics ground zero spotlight newsletter view india looking world affairs indian perspective see newsletters states cities entertainment section entertainment art dance movies music reviews theatre newsletter first day first show news reviews world cinema streaming see newsletters opinion section opinion editorial cartoon columns comment interview lead letters open page corrections clarifications business section business economy industry markets budget sport section sport cricket football hockey tennis athletics motorsport races sports topics wickets technology section technology gadgets internet newsletter today cache download top 5 technology stories day see newsletters science section science topics isro question corner newsletter sc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict


nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
folder_path = "/content/text_pages"
inverted_index = defaultdict(set)


for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
            lines = f.readlines()


            content = ''.join(lines[1:]).lower()

            tokens = word_tokenize(content)
            cleaned_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]


            for word in set(cleaned_tokens):
                inverted_index[word].add(filename)

print("Inverted Index created ✅")
print(f"Total unique keywords indexed: {len(inverted_index)}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Inverted Index created ✅
Total unique keywords indexed: 17498


In [15]:
def search(query):
    query_tokens = word_tokenize(query.lower())
    query_tokens = [word for word in query_tokens if word.isalnum() and word not in stop_words]

    results = defaultdict(int)

    for word in query_tokens:
        for file in inverted_index.get(word, []):
            results[file] += 1


    def sort_key(item):
        filename, score = item
        is_ece = filename.endswith(".ece.txt")
        return (not is_ece, -score)

    sorted_results = sorted(results.items(), key=sort_key)

    return sorted_results


In [17]:
import os

TEXT_DIR = "text_pages"

query = input("Enter search query: ")
results = search(query)

if results:
    print("\n🔍 Search Results:")
    for filename, score in results:
        file_path = os.path.join(TEXT_DIR, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                first_line = f.readline().strip()
                if first_line.startswith("URL:"):
                    url = first_line[4:].strip()
                else:
                    url = "URL not found"
        except Exception as e:
            url = f"Error reading file: {e}"

        print(f"{url} (matched {score} keyword{'s' if score > 1 else ''})")
else:
    print("No matches found.")


Enter search query: Kohli

🔍 Search Results:
URL_NOT_FOUND (matched 1 keyword)
https://www.thehindu.com/opinion/columns/ (matched 1 keyword)
https://timesofindia.indiatimes.com/ (matched 1 keyword)
URL_NOT_FOUND (matched 1 keyword)
URL_NOT_FOUND (matched 1 keyword)
URL_NOT_FOUND (matched 1 keyword)
https://www.thehindu.com/topic/Between_Wickets/ (matched 1 keyword)
URL_NOT_FOUND (matched 1 keyword)
